<a href="https://colab.research.google.com/github/jjfantini/humblBlogTutorials/blob/master/Py/Stock_Sentiment_obb_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install & Load Libraries

In [ ]:
!pip install openbb
!pip install langchain

In [ ]:
from openbb_terminal.sdk import openbb as obb

## Collect OpenAI API Key

In [5]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API')

## Collect Stock News Data

In [21]:
# Collect Option Chain Data

# News for OpenAI Analysis
news = obb.news(term="AAPL")

# Used for alternative news sources
# news2 = obb.stocks.news(term="AAPL", sort="articlePublishedDate").head(10)

# Bull/Bear Ratio Data
bullbear = obb.stocks.ba.bullbear("AAPL")

# Finbrain Sentiment (to plot a sentiment graph overtime)
stock_sentiment = obb.stocks.ba.headlines("AAPL")


## Create AI Stock Sentiment Agent

In [8]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

# Use gpt-3.5-turbo if you don't have gpt-4 access, although if you don't...where have you been lol
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=OPENAI_API_KEY)

### Create Agent Instructions

In [12]:
prompt = """
# Background
You are a qualified and experienced stock analyst.
You have been working in this field for over 20 years, and have gotten accustomed to trader behaviour.
This has allowed you to become proficient at gauging investor sentiment based on a news headline and small exerpts of news articles.
You will be given either a news headline or summary of the article.

# Tasks
1. You are to answer this question:
Is the predominant sentiment in the following statement positive, negative, or neutral?
---------
Statement: {statement}
---------

2. Respond with one word in lowercase: positive, negative, or neutral.

[your sentiment choice]
"""

chain = LLMChain.from_string(
    llm=llm,
    template=prompt
)

## Compile All Sentiment Data

### Generate Sentiment Analysis from News

In [13]:
# Create a table to collect your analysis results
import pandas as pd

sentiment_table = pd.DataFrame()
sentiment_table["news_headline"] = news['Description']
sentiment_table["sentiment"] = news.Description.apply(chain.run)

In [34]:
bullbear

(898648, 19, 4, 15)

In [79]:
## BULL BEAR RATIO CALCULATION
import numpy as np

def calculate_bullbear_ratio(bullbear):
    """
    Calculate the bull/bear ratio from a tuple containing bullish and bearish statement counts.
    Bearish: -1
    Neutral:  0
    Bullish:  1

    Parameters:
    bullbear (tuple): A tuple returned from `obb.stocks.ba.bullbear()`

    Returns:
    Float: The value of bulish/bearish sentiment.
    """

    # Extract the number of bullish and bearish statements from the tuple
    num_bullish = bullbear[2]
    num_bearish = bullbear[3]

    # Calculate the bull/bear ratio
    # To avoid division by zero, check if the total number of statements is not zero
    total = num_bullish + num_bearish
    if total != 0:
        bullbear_ratio = (num_bullish - num_bearish) / total
    else:
        bullbear_ratio = 0  # Assign zero if total count is zero

    print(f"Bullish/Bearish Ratio: {bullbear_ratio}")

    return bullbear_ratio


In [80]:
bullbear_ratio = calculate_bullbear_ratio(bullbear)


Bullish/Bearish Ratio: -0.5789473684210527


In [66]:
import plotly.express as px
import pandas as pd

# Create a simple line plot using Plotly
fig = px.line(stock_sentiment, y='Sentiment Analysis', title='Sentiment Analysis Over Time')

# Add a horizontal line at y=0 to emphasize the zero point
fig.add_hline(y=0, line_dash="dash", line_color="red")

# Show plot
fig.show()



### Future Directions


*   Add a filtering step to ensure the news_headlines column has 'AAPL'/'Apple' text in it.
* Aggregate all statistics to calculate one comprehensive bullish/bearish sentiment signal

